# Decipher range-v3 auto types

[range-v3](https://github.com/ericniebler/range-v3) is a range library which will (most probably) be included in the next C++ standard. 
It allows very expressive code. However, the design of the library is based on a complex collection of types, so that most of the functions and variables can only be noted as `auto`.  

`cleantype` can help in deciphering what's going on with the types with range-v3: 

First, we include cleantype and the range-v3 library

In [1]:
#pragma cling add_include_path("./include")
#include <cleantype/cleantype.hpp>
#include <range/v3/all.hpp>

#define run__show(...)                  \
{                                       \
    std::cout << #__VA_ARGS__ << "\n";  \
    std::cout << __VA_ARGS__ << "\n\n"; \
}
using namespace ranges;

Then we set the indent depth limit to 2, because indenting will make the types much easier to read, since `range-v3` uses long and deep nested types. 

In [2]:
cleantype::CleanConfiguration::GlobalConfig().indent_depth_limit = 2;

### Example of a view that return square numbers

Let's define a function that yields using `ranges::yield` and, then let's identify its return type.

In [3]:
auto square_yield_fn(int x) {
    return ranges::yield(x * x);
}
run__show(  CT_type_fn(square_yield_fn, int)  );

CT_type_fn(square_yield_fn, int)
ranges::v3::single_view<int>



Now, let's define a view that transform ints into squares. This view is lazy, and unlimited (it never ends). We then identify its type:

In [4]:
auto squares_view_fn = view::for_each(view::ints(1), square_yield_fn);
run__show(  cleantype::clean(squares_view_fn)  );

cleantype::clean(squares_view_fn)
ranges::v3::join_view
<
    ranges::v3::transform_view
    <
        ranges::v3::iota_view
        <
            int,
            void
        >,
        ranges::v3::single_view
        <
            int
        > (*)(int)
    >,
    void
> &



In [5]:
auto squares_fn_take_10 = squares_view_fn | view::take(10);
run__show(  cleantype::clean(squares_fn_take_10)  );

cleantype::clean(squares_fn_take_10)
ranges::v3::detail::take_exactly_view_
<
    ranges::v3::join_view
    <
        ranges::v3::transform_view
        <
            ranges::v3::iota_view
            <
                int,
                void
            >,
            ranges::v3::single_view
            <
                int
            > (*)(int)
        >,
        void
    >,
    false
> &



### Example with range-v3 and lambdas:

Since lambda are actually anonymous structs, cleantype cannot disclose the signature of the inner lambda of a view that is contructed using a lambda. 

In [6]:
{
    using namespace ranges;
    
    auto square_yield_lambda = [](int x) {
        return yield(x * x);
    };
    run__show(  cleantype::lambda_clean(square_yield_lambda)  );
    auto squares_view_lambda = view::for_each(view::ints(1), square_yield_lambda);
    run__show(  cleantype::clean(squares_view_lambda)  );
}

cleantype::lambda_clean(square_yield_lambda)
lambda: (int) -> ranges::v3::single_view<int>

cleantype::clean(squares_view_lambda)
ranges::v3::join_view
<
    ranges::v3::transform_view
    <
        ranges::v3::iota_view
        <
            int,
            void
        >,
        (lambda at input_line_14:5:32)
    >,
    void
> &



In an ideal world, I would be interesting to be able to display the view type as below (but I'm afraid that the code in order to get to this might be intractable).

````
ranges::v3::join_view
<
    ranges::v3::transform_view
    <
        ranges::v3::iota_view
        <
            int,
            void
        >,
        lambda: (int) -> ranges::v3::single_view<int>
    >,
    void
> &
````

Thus, it is advised to prefer "auto return functions" to lambdas when using range-v3 with cleantype.